In [ ]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from deepface import DeepFace
import cv2 
import PIL.Image as Image
# import tensorflow as tf
# tf.config.list_physical_devices('GPU') 

2024-10-15 05:42:56.066556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 05:42:56.078363: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 05:42:56.081955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 05:42:56.091188: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 05:42:56.836258: W tensorflow/compiler/tf2

In [11]:
# model_name = 'Facenet512'
# detector_backend = 'mtcnn'
# enforce_detection = False
# distance_metric = 'euclidean_l2'

model_name = 'ArcFace'
detector_backend = 'retinaface'
enforce_detection = False
distance_metric = 'cosine'

In [3]:
# step 1: using lfw-names.txt, filter people with more than 2 images
names = pd.read_csv('datasets/lfw-names.txt', sep='\t', header=None)
names[2] = [str(i).zfill(5) for i in range(len(names))]
names.to_csv('datasets/lfw_name_number_map.txt', sep='\t', header=None, index=None)

In [ ]:
# print intersection of names_2 and names_1
print(names.shape)
total_images = sum(names[1])
print(total_images)
print(len(names))

(5749, 3)


In [8]:
# step 2: create a train, test and unseen test split
os.mkdir('datasets/lfw')
os.mkdir('datasets/lfw/train')
os.mkdir('datasets/lfw/test')
split = 4500
for i in range(0, 4500):
    # add first image to unseen test set
    os.system('cp datasets/lfw_raw/'+names.iloc[i, 0]+"/"+names.iloc[i, 0]+"_0001.jpg datasets/lfw/train/"+names.iloc[i, 2]+".jpg")

for i in range(4500, len(names)): # to keep the same number of people (not necessary)
    os.system('cp datasets/lfw_raw/'+names.iloc[i, 0]+"/"+names.iloc[i, 0]+"_0001.jpg datasets/lfw/test/"+names.iloc[i, 2]+".jpg")


In [ ]:
d2 = f'lfw_{model_name.lower()}_pp'

os.mkdir(f'datasets/{d2}')
os.mkdir(f'datasets/{d2}/train')
os.mkdir(f'datasets/{d2}/test')
os.mkdir(f'datasets/{d2}/train/images')
os.mkdir(f'datasets/{d2}/test/images')
os.mkdir(f'datasets/{d2}/train/embeddings')
os.mkdir(f'datasets/{d2}/test/embeddings')

In [13]:
dir = 'datasets/lfw/train'
for file in os.listdir(dir):
    if os.path.exists(f'datasets/{d2}/train/images/' + file):
        print('Skipping', file)
        continue
    img_path = dir+'/'+file
    emb = DeepFace.represent(img_path, model_name=model_name, detector_backend=detector_backend, enforce_detection=enforce_detection, align=True)
    embedding = emb[0]['embedding']
    face_area = emb[0]['facial_area']
    face = Image.open(img_path)
    x, y, w, h = face_area['x'], face_area['y'], face_area['w'], face_area['h']
    face = face.crop((x, y, x+w, y+h))
    face = face.resize((160, 160))
    face.save(f'datasets/{d2}/train/images/' + file)
    np.save(f'datasets/{d2}/train/embeddings/' + file.split('.')[0] + '.npy', embedding)

dir = 'datasets/lfw/test'
for file in os.listdir(dir):
    if os.path.exists(f'datasets/{d2}/test/images/' + file):
        print('Skipping', file)
        continue
    img_path = dir+'/'+file
    emb = DeepFace.represent(img_path, model_name=model_name, detector_backend=detector_backend, enforce_detection=enforce_detection, align=True)
    embedding = emb[0]['embedding']
    face_area = emb[0]['facial_area']
    face = Image.open(img_path)
    x, y, w, h = face_area['x'], face_area['y'], face_area['w'], face_area['h']
    face = face.crop((x, y, x+w, y+h))
    face = face.resize((160, 160))
    face.save(f'datasets/{d2}/test/images/' + file)
    np.save(f'datasets/{d2}/test/embeddings/' + file.split('.')[0] + '.npy', embedding)


W0000 00:00:1728972993.337361   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.338428   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.339510   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.340506   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.341575   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.342894   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.344026   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.345445   52261 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728972993.346941   52261 gp